**Read Bfore Running the cells:**
- Run on Firefox
- This is specifically for initial testing files only
-It'll create a new entry in the TileDB database and attach a diary file inside it, which is the .txt file you will be uploading
- It will also create a note to write down that it's Initial testing

In [ ]:
#Uncomment this part if not already installed

'''
!apt update
!apt install -y firefox
!wget -q https://github.com/mozilla/geckodriver/releases/latest/download/geckodriver-linux64.tar.gz
!tar -xvzf geckodriver-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/
'''

In [ ]:
#Uncomment this part if not already installed
'''
!apt update > /dev/null
!apt install -y firefox-esr > /dev/null
!wget https://github.com/mozilla/geckodriver/releases/download/v0.33.0/geckodriver-v0.33.0-linux64.tar.gz > /dev/null
!tar -xvzf geckodriver-v0.33.0-linux64.tar.gz > /dev/null
!mv geckodriver /usr/local/bin/
!pip install selenium > /dev/null
'''


In [ ]:
import os
import time
import traceback
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
def create_logged_in_session(username, password):
    options = Options()
    options.add_argument('--headless')  # No browser window
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    cli = webdriver.Firefox(options=options)
    cli.get("https://www.particle.cz/work/TileDB2/account/login")

    try:
        WebDriverWait(cli, 15).until(
            EC.presence_of_element_located((By.ID, "MainContent_loginEmailTxt"))
        )
        cli.find_element(By.ID, "MainContent_loginEmailTxt").send_keys(username)
        cli.find_element(By.ID, "MainContent_loginPasswordTxt").send_keys(password)
        cli.find_element(By.ID, "MainContent_LoginBtn").click()

        WebDriverWait(cli, 15).until(
            EC.url_contains("TileDB2")
        )
        print("[✅] Logged in successfully.")
    except Exception as e:
        print("[❌] Login failed.")
        cli.save_screenshot("login_error.png")
        with open("login_error.html", "w", encoding="utf-8") as f:
            f.write(cli.page_source)
        raise e

    return cli


In [ ]:
import getpass

username = input("▶Your TileDB Username Pretty Please, then Enter: ")
password = getpass.getpass("👨🏻‍💻Your Password (Don't worry, no one sees it) (input hidden): ")

cli = create_logged_in_session(username, password)



▶Your TileDB Username Pretty Please, then Enter: thabo.pilusa@cern.ch
👨🏻‍💻Your Password (Don't worry, no one sees it) (input hidden): ··········


[✅] Logged in successfully.


In [ ]:
def batch_register(cli, base_path="Burn_In"):
    import os
    import time
    import traceback
    from datetime import datetime
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import re

    os.makedirs("screenshots", exist_ok=True)
    os.makedirs("html_dumps", exist_ok=True)

    for file_name in sorted(os.listdir(base_path)):
        if not file_name.endswith(".txt"):
            continue

        try:
            name_without_ext = file_name[:-4]
            parts = name_without_ext.split()

            if len(parts) < 2:
                print(f"[⚠️] Could not extract date from '{file_name}'. Using today's date.")
                date_obj = datetime.today()
            else:
                date_str = parts[1].replace("-", ".")
                date_obj = datetime.strptime(date_str, "%m.%d.%Y")

            # Extract numeric Brick ID (3–4 digits) from filename like 'SA-100' or 'SA-1000'
            match = re.search(r'(\d{3,4})', parts[0])
            if not match:
                print(f"[⚠️] Could not find numeric Brick ID in '{file_name}'")
                continue
            suffix = match.group(1)
            brick_id = f"ISA{suffix}"

            print(f"\nRegistering: Brick '{brick_id}' from '{file_name}'")

            # Manufacture new Brick page
            cli.get("https://www.particle.cz/work/TileDB2/brick10v/manufacture")
            WebDriverWait(cli, 15).until(EC.presence_of_element_located((By.ID, "MainContent_brickCodeTxt")))

            # Uncheck prefix checkbox
            try:
                prefix_checkbox = cli.find_element(By.ID, "MainContent_brickCodePrefixEnabled")
                if prefix_checkbox.is_selected():
                    prefix_checkbox.click()
                    print("[🧹] Unchecked 'Include code prefix'")
            except:
                print("[⚠️] Could not uncheck prefix checkbox")

            # Fill Brick ID
            cli.find_element(By.ID, "MainContent_brickCodeTxt").send_keys(brick_id)

            # Location
            location_input = cli.find_element(By.ID, "MainContent_brickLocationCB_brickLocationCB_TextBox")
            location_input.clear()
            location_input.send_keys("Wits")
            time.sleep(1)
            location_input.send_keys(Keys.TAB)

            # Manufacture Date
            formatted_date = date_obj.strftime("%d.%m.%Y 00:00 AM")
            date_input = cli.find_element(By.ID, "MainContent_brickCreatedTxt")
            date_input.clear()
            date_input.send_keys(formatted_date)
            date_input.send_keys(Keys.TAB)

            # Note = filename
            note_input = cli.find_element(By.ID, "MainContent_brickNoteTxt")
            note_input.clear()
            note_input.send_keys(f"Data file: {file_name}")

            cli.save_screenshot(f"screenshots/{brick_id}_before_submit.png")

            # Submit the form
            save_btn = WebDriverWait(cli, 10).until(
                EC.element_to_be_clickable((By.ID, "MainContent_brickSaveBtn"))
            )
            save_btn.click()
            time.sleep(2)

            # Handle alert if any
            try:
                WebDriverWait(cli, 3).until(EC.alert_is_present())
                alert = cli.switch_to.alert
                print(f"[⚠️] Alert: {alert.text}")
                alert.accept()
            except:
                print(f"[✅] Brick '{brick_id}' registered.")

            # After registration, upload .txt file as diary record
            upload_diary_record(cli, brick_id, os.path.join(base_path, file_name))

        except Exception as e:
            print(f"[❌] Error with '{file_name}': {e}")
            cli.save_screenshot(f"screenshots/error_{brick_id}.png")
            with open(f"html_dumps/error_{brick_id}.html", "w", encoding="utf-8") as f:
                f.write(cli.page_source)
            traceback.print_exc()


In [ ]:
def upload_diary_record(cli, brick_id, txt_file_path):
    import os
    import time
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    try:
        print(f"📎 Attaching diary file for '{brick_id}'")

        # Go to the measure page for this brick
        cli.get(f"https://www.particle.cz/work/TileDB2/brick10v/measure?partcode={brick_id}")
        WebDriverWait(cli, 10).until(EC.presence_of_element_located((By.ID, "MainContent_measureFileUpl")))

        # Upload file
        upload_input = cli.find_element(By.ID, "MainContent_measureFileUpl")
        upload_input.send_keys(os.path.abspath(txt_file_path))

        # Select author
        author_box = cli.find_element(By.ID, "MainContent_measureAuthorCB_measureAuthorCB_TextBox")
        author_box.clear()
        author_box.send_keys("Thabo Pilusa")
        time.sleep(1)
        author_box.send_keys(Keys.TAB)

        # Note: set note to "Passed Final Testing"
        note_area = cli.find_element(By.ID, "MainContent_measureNoteTxt")
        note_area.clear()
        note_area.send_keys("Passed Initial Testing")

        # Optional: you can adjust the date too if needed
        date_input = cli.find_element(By.ID, "MainContent_brickMeasuredTxt")
        date_input.clear()
        date_input.send_keys(datetime.now().strftime("%d.%m.%Y %H:%M"))
        date_input.send_keys(Keys.TAB)

        # Submit
        save_btn = cli.find_element(By.ID, "MainContent_brickMeasureBtn")
        save_btn.click()
        time.sleep(2)

        print(f"[✅] Diary file uploaded for '{brick_id}'.")

    except Exception as e:
        print(f"[⚠️] Failed to upload diary for '{brick_id}': {e}")


In [ ]:
#Insert your own Directory to your input files
#Files must be in this format usually from the testing stations: SA-001 8-23-2024 1-19 PM
batch_register(cli, base_path="Final_Testing")